In [18]:
import pandas as pd
import numpy as np

In [19]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [20]:
df1 = pd.read_csv("data/all_models_metrics_for_paper.csv").fillna("")

In [21]:
df1["model_temp"] = df1["model"] + " (T=" + df1["temperature"].astype(str) + ")"

In [22]:
df1.columns

Index(['model', 'dataset_size', 'temperature', 'fine_tuned', 'total_clauses',
       'total_found', 'total_cut_clauses', 'total_embedded', 'found_embedded',
       'total_non_embedded', 'found_non_embedded', 'found_cut_clauses',
       'total_other_clauses', 'found_other_clauses', 'total_non_cut_clauses',
       'found_non_cut_clauses', 'total_verses', 'correct_verses',
       'simple_correct', 'complex_correct', 'total_simple_clauses',
       'total_simple_found', 'total_complex_clauses', 'total_complex_found',
       'model_temp'],
      dtype='object')

In [23]:
df2 = pd.read_csv("data/clause_number_results_assessment.csv")

In [24]:
df1.head(2)

,model,dataset_size,temperature,fine_tuned,total_clauses,total_found,total_cut_clauses,total_embedded,found_embedded,total_non_embedded,found_non_embedded,found_cut_clauses,total_other_clauses,found_other_clauses,total_non_cut_clauses,found_non_cut_clauses,total_verses,correct_verses,simple_correct,complex_correct,total_simple_clauses,total_simple_found,total_complex_clauses,total_complex_found,model_temp
0,GPT-4o mini,medium,0,no,3827,1444,114,127,7,3700,1437,0,3586,1437,3713,1444,1000,209,209,0,3301,1350,526,94,GPT-4o mini (T=0)
1,GPT-4o mini,large,0,no,10204,3818,306,341,18,9858,3800,0,9553,3800,9898,3818,2616,527,527,0,8805,3560,1399,258,GPT-4o mini (T=0)


In [25]:
df2.head(2)

,category,False,True,percentage,model_temp,size,status
0,complex,286,0,0.0,GPT-4o mini (T=0),large,base
1,simple_1_clause,237,27,10.2,GPT-4o mini (T=0),large,base


In [38]:
# --- 1) Prep metadata to match df2 ---
df = df1.copy()

df["model_temp"] = df["model"].astype(str) + " (T=" + df["temperature"].astype(str) + ")"
df["size"] = df["dataset_size"]

# Map fine_tuned -> status like df2 ("base" vs "fine-tuned")
status_map = {"no": "base", "yes": "fine-tuned", "false": "base", "true": "fine-tuned", "0": "base", "1": "fine-tuned"}
df["status"] = (
    df["fine_tuned"].astype(str).str.strip().str.lower().map(status_map).fillna(df["fine_tuned"])
)

id_vars = ["model_temp", "size", "status"]

# --- 2) Explicit pairs for the requested categories only ---
pairs = {
    "other_clauses":   ("found_other_clauses", "total_other_clauses"),
    "embedded_clauses":("found_embedded",      "total_embedded"),
    "cut_clauses":     ("found_cut_clauses",   "total_cut_clauses"),
}

# Optional: validate columns exist
missing = [col for (f,t) in pairs.values() for col in (f,t) if col not in df.columns]
if missing:
    raise KeyError(f"Missing expected columns in df1: {missing}")

# --- 3) Stack into long with True / False / percentage ---
frames = []
for category, (found_col, total_col) in pairs.items():
    sub = df[id_vars + [found_col, total_col]].copy()
    sub["category"] = category

    # ensure numeric
    sub[found_col] = pd.to_numeric(sub[found_col], errors="coerce")
    sub[total_col] = pd.to_numeric(sub[total_col], errors="coerce")

    sub = sub.rename(columns={found_col: "True", total_col: "Total"})
    sub["False"] = sub["Total"] - sub["True"]
    sub["percentage"] = np.where(
        sub["Total"] > 0,
        np.round((sub["True"] / sub["Total"]) * 100, 1),
        0.0
    )

    frames.append(sub)

df_long = pd.concat(frames, ignore_index=True)

# --- 4) Final column order to mirror df2 ---
df_long = df_long[["category", "False", "True", "percentage", "model_temp", "size", "status"]]

# Optional: sort for readability (preserves your three categories grouped per model/size/status)
df_long = df_long.sort_values(["model_temp", "size", "status", "category"], ignore_index=True)

In [43]:
df_long.columns

Index(['category', 'False', 'True', 'percentage', 'model_temp', 'size',
       'status'],
      dtype='object')

In [41]:
df_long.to_csv("data/clause_categories_models_results.csv", index=False)